# Perform Sentiment Analysis on Reddit Posts

Reddit API:

Go to https://www.reddit.com/prefs/apps
Click "create application"
Name: My Sentiment Example
Choose script
Leave description and about url blank
Add http://www.example.com/unused/redirect/uri to redirect uri

### Library Imports

In [13]:
import os
import praw
import openai

In [14]:
reddit_client_id = os.environ["REDDIT_CLIENT_ID"]
reddit_secret_key = os.environ["REDDIT_SECRET_KEY"]

openai.api_key = os.environ["OPENAI_API_KEY"]

In [15]:
'''
Initialize the Reddit API
'''
reddit = praw.Reddit(client_id=reddit_client_id,
                     client_secret=reddit_secret_key,
                     user_agent="Sentiment Analysis test by suvofalcon")

In [16]:
# We will access hot subreddits
subreddit_cricket = reddit.subreddit("cricket")
print(f"Display name for the subreddit - {subreddit_cricket.display_name}")
print(f"Title Name of the Subreddit - {subreddit_cricket.title}")
print(f"Number of accounts active on this subreddit - {subreddit_cricket.accounts_active}")

Display name for the subreddit - cricket
Title Name of the Subreddit - Cricket
Number of accounts active on this subreddit - 4871


In [49]:
# Function to create a list of comments for a given topic
def create_list_of_comments(topic, num_of_hot_topics, num_of_comments):
    list_comments = []
    for post in reddit.subreddit(topic).hot(limit=num_of_hot_topics):
        print (f"Title - {post.title}")
        print("\n")
        submission = reddit.submission(post.id)
        counter = 0
        for comment in submission.comments:
            comment = comment.body
            if not comment == "[deleted]":
                list_comments.append(comment)
                counter += 1
            if counter == num_of_comments:
                break
    return list_comments

In [53]:
# We will create a list of comments by making the function call 
list_comments = create_list_of_comments('cricket', 3, 2)
# We will check the list
print(list_comments)
print("\n")
print(f"Number of elements in the list - {len(list_comments)}")

Title - Daily General Discussion and Match Links Thread - October 06, 2023

Title - ICC Cricket World Cup Hub- Group Stage Thread

Title - Afghanistan beat Pakistan by 4 wickets to set up Asian Games cricket final with India

['Belated thoughts from last night’s game:\n\nI’m feeling unbelievably smug that the entire world is now beating the ‘Ravindra needs to bat up the order’ drum. He’s serviceable as a lower order slogger and part-time bowler, but we have other guys who can do that better. As we saw yesterday, Ravindra’s talents lie elsewhere \n\nI can’t really believe we got away with three frontline bowlers against an extra-stacked England batting lineup. A lot of stuff had to go right for that to work. None of the frontliners could afford to have anything other than a really good performance, and all 3 nailed it. Boult/Henry/Santner are top bowlers, but we can’t realistically expect them all to turn in <5RPO plus wickets every single match, so I don’t think this strategy is going 

In [56]:
# Create a function to arrange the comments in bullets
def arrange_comments(list_of_comments):
    str_comments = ''
    counter = 1
    for index in list_of_comments:
        str_comments = str_comments + f"{counter}. {index} \n"
        counter +=1
    return str_comments

In [58]:
arranged_comments = arrange_comments(list_comments)
print(arranged_comments)

1. Belated thoughts from last night’s game:

I’m feeling unbelievably smug that the entire world is now beating the ‘Ravindra needs to bat up the order’ drum. He’s serviceable as a lower order slogger and part-time bowler, but we have other guys who can do that better. As we saw yesterday, Ravindra’s talents lie elsewhere 

I can’t really believe we got away with three frontline bowlers against an extra-stacked England batting lineup. A lot of stuff had to go right for that to work. None of the frontliners could afford to have anything other than a really good performance, and all 3 nailed it. Boult/Henry/Santner are top bowlers, but we can’t realistically expect them all to turn in <5RPO plus wickets every single match, so I don’t think this strategy is going to be very sustainable

GP’s performance has gone largely unnoticed but he was pretty close to bowling MVP for me. Took two huge wickets at huge times: Moeen when they still had a lot of left-arm spin to get through, and getting 

### Prompt Generation

First we will create a function which generates a prompt

In [59]:
def create_prompt(arranged_comments):
    prompt = (f"Give the Sentiment of each of the comments below as either 'Positive', 'Negative' or 'Neutral' \n"
              f"{arranged_comments}")
    return prompt

In [60]:
print(create_prompt((arranged_comments)))

Give the Sentiment of each of the comments below as either 'Positive', 'Negative' or 'Neutral' 
1. Belated thoughts from last night’s game:

I’m feeling unbelievably smug that the entire world is now beating the ‘Ravindra needs to bat up the order’ drum. He’s serviceable as a lower order slogger and part-time bowler, but we have other guys who can do that better. As we saw yesterday, Ravindra’s talents lie elsewhere 

I can’t really believe we got away with three frontline bowlers against an extra-stacked England batting lineup. A lot of stuff had to go right for that to work. None of the frontliners could afford to have anything other than a really good performance, and all 3 nailed it. Boult/Henry/Santner are top bowlers, but we can’t realistically expect them all to turn in <5RPO plus wickets every single match, so I don’t think this strategy is going to be very sustainable

GP’s performance has gone largely unnoticed but he was pretty close to bowling MVP for me. Took two huge wick

In [61]:
# Pass to Open AI
response = openai.Completion.create(engine="text-davinci-003",
                                    prompt = create_prompt(arranged_comments),
                                    max_tokens=256,
                                    temperature=0,
                                    top_p=1.0)
response

<OpenAIObject text_completion id=cmpl-86fxJjigQNG17CYo22WKznDiUUL3X at 0x10fa046b0> JSON: {
  "warning": "This model version is deprecated. Migrate before January 4, 2024 to avoid disruption of service. Learn more https://platform.openai.com/docs/deprecations",
  "id": "cmpl-86fxJjigQNG17CYo22WKznDiUUL3X",
  "object": "text_completion",
  "created": 1696601937,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": "\n1. Neutral\n2. Neutral\n3. Neutral\n4. Positive\n5. Neutral\n6. Negative",
      "index": 0,
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 524,
    "completion_tokens": 24,
    "total_tokens": 548
  }
}

In [62]:
# Check the response text
response['choices'][0]['text']

'\n1. Neutral\n2. Neutral\n3. Neutral\n4. Positive\n5. Neutral\n6. Negative'